In [ ]:
#Определение пропусков
def nan (df):
    df_na = pd.DataFrame(df.isna().sum())
    df_na['percent'] = df_na[0] * 100 / len(df)
    df_na.columns = ['count', 'percent']
    return df_na[(df_na['percent'] > 0)].sort_values('percent', ascending = False)

#Диаграмма донат
def bublik(data, areas, index):
    areas =  areas
    for i in areas:
        data.pivot_table(index = index, values = [i],
          aggfunc = 'sum').sort_values([i], ascending = False).plot(y = i, 
          wedgeprops=dict(width=0.3), shadow=False, kind = 'pie', rotatelabels=True,
          label = i, autopct='%1.0f%%').legend(bbox_to_anchor=(1.1, 1))    
        
#Диаграмма boxplot_hue
def boxplot(data, x, y, hue):
    plt.figure(figsize=(15, 5))
    sns.boxplot(x=x, y=y, hue=hue,
                 data=telecom, palette='Set3')
    plt.show()

#Просто график
def graph (data, x, y, name):
    plt.figure(figsize=(15, 10))
    sns.relplot(x=x, y=y, kind="line", data=data, height=5, aspect=3)
    plt.title(name, fontsize=16)
    plt.show()    
    
#Гистограмма датасета    
def hist (data):
    fig = plt.figure(figsize = (15,15))
    ax = fig.gca()
    data.hist(ax = ax)
    plt.show()
    
#Разделение выборок 
def split(data, target, first_size, second_size):
    features = data.drop(target, axis=1)
    target = data[target]
                 
    features_other, features_test, target_other, target_test = train_test_split(
    features, target, test_size=first_size, random_state=42)
    features_train, features_valid, target_train, target_valid = train_test_split(
    features_other, target_other, test_size=second_size, random_state=42)
    
    print('features_train', len(features_train))
    print('features_valid', len(features_valid))
    print('features_test', len(features_test))
    
    return features_test, target_test, features_train, target_train, \
    features_valid, target_valid

#Даунсемплинг
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=42)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=42)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    print('Баланс класса: 0 - ', round(target_downsampled.value_counts()[0] / (target_downsampled.value_counts().sum()/100)), '%')
    return features_downsampled, target_downsampled

#Апсэмплинг
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=42)
    
    print('Баланс класса: 0 - ', round(target_upsampled.value_counts()[0] / (target_upsampled.value_counts().sum()/100)), '%')
    return features_upsampled, target_upsampled

#Dалидация
def cv(model, X, y, scoring, cv):
    scores = cross_validate(model, X, y, cv=cv, 
                        scoring=(scoring),
                        return_train_score=False)
   
    return pd.DataFrame(scores)